In [2]:
from transformers import file_utils
# print(file_utils.default_cache_path)
# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'garbage_collection_threshold:0.6,max_split_size_mb:512'
import transformers
import torch
from huggingface_hub import login
login(token = '')


model_id = "meta-llama/Meta-Llama-3-8B-instruct"

# Create a text generation pipeline that uses the model above.
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",pad_token_id = 50256
)




Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/vmonjezi/.cache/huggingface/token
Login successful


2024-05-07 20:34:36.252353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 20:34:37.214961: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/home/vmonjezi/.virtualenvs/RISC_LAB/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/vmonjezi/.virtualenvs/RISC_LAB/lib/python3.8/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
messages = [
#     {"role": "system", "content": "A definition for a counterfactual of an individual. Who among us has never wondered \
#      about what might have been had some past choice been different? With perhaps a little more effort, you might have \
#      been an athlete, a doctor, maybe even a rock star. Who among us has never regretted choices made and actions taken? \
#      Maybe you should have studied harder in school, traveled more when you had the chance, or had the salmon for lunch \
#      instead of the pasta. And who has never pondered a lost love and imagined how passionate it might have been? \
#      Thinking about what might have been, about alternatives to our own pasts, is central to human thinking and emotion.\
#      Such thoughts are called counterfactual thoughts.\
#      Consider that mwn an women have different distribution of features. For example \
#      a 140K income for men would be a different value for women as women have less income in average. So consider this \
#      when imagining the counterfacual.\
#      In reality, the gender pay gap is complex and influenced by\
#      many factors, including systemic biases, discrimination, and differences in career choices and opportunities. But use \
#      your knowledge of the internet to estimate the gap for different features!\
#      So you are a counterfacual estimater. I will provide some features and then\
#     you just estimate the counterfacual considering the gaps for diffrent features. Do not provide any reasoning.\
#     Just give the features of the counterfactual the same way \
#      I provide the real-world individual features."},
    
    # Telling the model how to behave and how to respond to the user when given information.
    # Describing it's main purpose without much detail on how it got it's conclusion.
    {"role": "system", "content": " you are a counterfacual estimator of the individual that user gives. \
    The user provides the features of the real-world individual. Then you estimate the counterpart based on\
    the requested feature like sex, race, age, and etc. do not provide any explanation on how you estimate, \
    just give the features of the counterpart same as the provided features by the user." },

    # User requesting counterfactual estimates.
    {"role": "user", "content":"I am Male, 42, income 80K. what would be my 41 years old counterfactual?"},
    
]

# torch.cuda.empty_cache()
# import gc

# gc.collect()
# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'garbage_collection_threshold:0.6,max_split_size_mb:32'

# Converts the conversation in messages into a single text prompt to give to the llm
prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Generates the prompts response.
outputs = pipeline(
    prompt,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    top_p=0.99,
)
print(outputs[0]["generated_text"][len(prompt):])


Male, 41, income 70K


In [1]:
!pip install code-bert-score

  Preparing metadata (setup.py) ... done
  Created wheel for code-bert-score: filename=code_bert_score-0.4.1-py3-none-any.whl size=22072 sha256=a7955d6e1b58644945d71e410554b311f8d5cca22f5b134f0c46ef8c300e7a85
  Stored in directory: /home/vmonjezi/.cache/pip/wheels/3d/6a/61/fd0b91c781898d2199cc7183195a47fceffed45d443e6c8382
Successfully built code-bert-score
